#####Practice Data Enrichment 

######Adding new column and using split to derive the date field

In [0]:
from pyspark.sql.functions import *
#lets assume we have input file as "custsmodified_25/12/30.csv" and we derive the date field from the file name using split function as shown below.
assumed_file_name = "custsmodified_25/30/12.csv"
derived_date = assumed_file_name.split('_')[1].split('.')[0]
print(derived_date)
data_enrichdf = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/BB_Practice data/custsmodified_target/part-00000-tid-4632942997489778150-17e46c3c-26b5-4d02-b361-af59b8243fb9-329-1-c000.csv",mode="permissive",header=True)
display(data_enrichdf)
data_enrichdf.printSchema()

#using withColumn
data_enrichdf1 = data_enrichdf.withColumn("datadt",lit(derived_date)).withColumn("loaddt",current_date())

#using withColumns
data_enrichdf1 = data_enrichdf.withColumns({"datadt":lit(derived_date),"loaddt":current_date()})

#using select
data_enrichdf1 = data_enrichdf.select("*",lit(derived_date).alias("datadt"),current_date().alias("loaddt"))
display(data_enrichdf1)

#using selectExpr
data_enrichdf1 = data_enrichdf.selectExpr("*",f"'{derived_date}' as datadt","current_date() as loaddt")
data_enrichdf1.printSchema()
display(data_enrichdf1)

######Deriving a column uses substring and Renaming. 
withColumn/sRenamed()

In [0]:
#Derive column using withColumn
data_enrichdf2 = data_enrichdf1.withColumn("profflag",substring("prof",1,1))

#using select
data_enrichment2 = data_enrichdf1.select("*",substring("prof",1,1).alias("profflag"))

#using selectExpr
data_enrichdf2 = data_enrichdf1.selectExpr("*", "substring(prof,1,1) as profflag")
display(data_enrichdf2)

#Renaming column using withColumnRenamed
data_enrichdf3 = data_enrichdf2.withColumnRenamed("sourcesystem","sourcename")

#using withColumnsRenamed
data_enrichdf3 = data_enrichdf2.withColumnsRenamed({"sourcesystem":"sourcename","prof":"profession"})
display(data_enrichdf3.take(20))

######Modify/replace and remove/eliminate
using concat()

In [0]:
data_enrichdf4 = data_enrichdf3.withColumn("profession",col("sourcename"))
display(data_enrichdf4.take(10))

#modify column using withColumn
data_enrichdf4 = data_enrichdf3.withColumn("profession",concat("profession",lit('-'),"profflag"))

#using select
data_enrichdf4 = data_enrichdf3.select("custid","firstname","lastname",concat("profession",lit('-'),"profflag").alias("profession"),"age","sourcename","datadt","loaddt","profflag")

#using selectExpr
data_enrichdf4 = data_enrichdf3.selectExpr("custid","firstname","lastname","concat(profession,'-',profflag) as profession","age","sourcename","datadt","loaddt","profflag")
display(data_enrichdf4.take(10))

#Remove column using withColumn
data_enrichdf5 = data_enrichdf4.drop("profflag")
display(data_enrichdf5.take(10))

#python code can be used to add column and assign a value
#store = "BB wholesale"
#expression = f"'{store}' as store"
#data_enrichdf5.selectExpr("*",expression).display()

 

######Split & Merge and Format & casting

In [0]:
#split using withColumn
splitdf = data_enrichdf5.withColumn("profflag",split("profession",'-'))
display(splitdf) # this wil store value in list format as in column profflag:["Pilot","P"]

#to seperate flag from actual prof value we can use index value:
splitdf1 = splitdf.withColumn("profession",col("profflag")[0])
display(splitdf1) # this will assign profession column with pilot alone
#or
#alternate way to split and assign correct value:
splitdf1 = splitdf.withColumn("profession",col("profflag").getItem(0))
display(splitdf1) 

splitdf2 = splitdf1.withColumn("profflag",col("profflag")[1])

splitdf3 = splitdf2.withColumn("shortprof",upper(substring(col("profession"),1,3)))

#merge using withColumn
mergeddf1 = splitdf3.select(
    col("custid"),
    concat(
        col("firstname"),
        lit(" "),
        col("lastname")
    ).alias("fullname"),
    "profession", "age", "sourcename", "datadt", "loaddt", "profflag"
)
display(mergeddf1)

dedupdf = mergeddf1.distinct().orderBy("custid")
dedupdf = dedupdf.dropDuplicates(['custid'])
display(dedupdf)

#formatteddf = dedupdf.withColumn("datadt",col("datadt").cast("date")).display() # this didnt work, it says data is malformed and so casting cant be done

formatteddf = dedupdf.withColumn("datadt",to_date(col("datadt"),'YY/DD/MM'))
display(formatteddf)# this isnt giving right output need to check

formatteddf.printSchema()
formatteddf.write.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/BB_Practice data/custsmodified_DEnriched",mode="overwrite",header=True)

######